In [1]:
# Libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import statsmodels as sm 
import plotly.express as px
import plotly.io as pio
from pandas_profiling import ProfileReport
from matplotlib import colors
import folium
import folium.plugins
import branca.colormap as cmp
from math import radians, cos, sin, asin, sqrt
sns.set_style('darkgrid')
pio.templates.default = 'seaborn'

ModuleNotFoundError: No module named 'pandas_profiling'

In [ ]:
df = pd.read_csv('../data/listings.csv')

In [ ]:
df.info()
#13621 observation and 18 variables
#note the neighbourhood_group has al it entries as null value 

In [ ]:
#dropping the neighbourhood_group 
df.drop('neighbourhood_group', axis =1, inplace=True)

In [ ]:
#512 differnt licenses, and only 881 out 13621 observation have value, so we will drop it. 
len(df.license.unique()) ,df.license.unique()
df.drop('license',axis =1, inplace=True)

In [ ]:
df.dtypes
#id, id_host are int variabe,most likely they are not used in graph or any mathematical operation
#I will change them to string type.
df['id'] = df['id'].astype(str)
df['host_id'] = df['host_id'].astype(str)
df.dtypes

In [ ]:
df[df['price'] >= 4000].count()['id']

In [ ]:
df.describe()
#the maximum of minimum nights looks like an error, so we will remove it.
#Also, we are only interested in listing with price less than 4000 $
df = df[df['minimum_nights']< 11000]
df = df[df['price'] < 4000]



In [ ]:
#df.neighbourhood.unique()
print(df['neighbourhood'].nunique())

In [ ]:
#The variation in price is high, the distribution of prices is:
#The distribution of price seems to be right skewed.

fig = px.histogram(
    df, 
    x='price', 
    template= 'ggplot2',
    title="Price Histogram",
    labels={
        'count': 'Count', #this doesnt work 
        'price': 'Price (CAD)'}
    )

fig.update_layout(yaxis_title='Count')
fig.show()

In [ ]:
fig = px.scatter( 
    data_frame=df[df['minimum_nights'] < 60],
    y='minimum_nights',
    x='number_of_reviews',
    template='ggplot2',
    opacity=0.2,
    labels= {
        'minimum_nights': 'Minimum Nights',
        'number_of_review': 'Number of Reviews'
    },
    title="Number of review vs. Minimum nights"
)
fig.show()

In [ ]:
fig = px.scatter( 
    data_frame=df[df['minimum_nights'] < 60],
    x='availability_365',
    y='number_of_reviews',
    template='ggplot2',
    opacity=0.15
)
fig.show()

In [ ]:
fig = px.histogram(
    df[df['minimum_nights'] <=181],
    x='minimum_nights',
    labels={
        'count': 'Count',
        'minimum_nights': 'Minimum Nights '
    },
    title='Minimum Nights Histogram',
    template='ggplot2'
    )
fig.update_layout(yaxis_title='Count')
fig.show()

# Introducing the parks datasets

In [ ]:
df3 = pd.read_csv('../data/lieux_d_interet.csv') #load the data
df3.dropna(how='any', inplace=True) #drop any missing value
df3 = df3[df3['Type'].isin(["Parc", "Jardin communautaire"])] #pick only places of interest related parks
df3 #only 4 parks

## Now mergin the new dataset it with original data set.
I will include two new columns to the original dataset; nerest park, and distance to the nearest park. 

In [ ]:
obs  = list(zip(df3['Latitude'], df3['Longitude']))
radians(obs[1][0])
len(obs)

In [ ]:
obs  = list(zip(df3['Latitude'], df3['Longitude']))
def cal_dist1(temp):
    lat1 = temp['latitude'] 
    lon1 = temp['longitude']
    lat1 = radians(lat1)
    lon1 = radians(lon1)
    results= []
    for i in range(len(obs)):
        lon2 = radians(obs[i][1])
        lat2 = radians(obs[i][0])
        # Haversine formula
        dlon = lon2 - lon1
        dlat = lat2 - lat1
        a = sin(dlat / 2)**2 + cos(lat1) * cos(lat2) * sin(dlon / 2)**2
        c = 2 * asin(sqrt(a))
        # Radius of earth in kilometers. Use 3956 for miles
        r = 6371
        # calculate the result
        results.append(c * r * 1.609344)
    
    min_val = min(results)
    if min_val == results[0]:
        return [min_val, "Park 1"]
    elif min_val == results[1]:
        return [min_val, "Park 2"]
    elif min_val == results[2]:
        return [min_val, "Park 3"]
    elif min_val == results[3]:
        return [min_val, "Park 4"]
    else:
        return [0,"0"]

df['distance_to_parks'] = df.apply(cal_dist1, axis=1)
df[['distance_to_nearest_park', 'nearest_park',]] = pd.DataFrame(df.distance_to_parks.tolist(), index=df.index)
df.drop('distance_to_parks', axis=1,inplace=True)
df.nearest_park.unique()

In [ ]:
df.distance_to_nearest_park.describe()
fig = px.histogram(
    df,
    x='distance_to_nearest_park',
    labels={'distance_to_nearest_park': 'Distance to nearest park (km)'},
    title='Distance to nearest park distribution.',
    template='ggplot2'
)
fig.update_layout(yaxis_title='Count')
fig.show()

In [ ]:
fig = px.bar(
    df,
    x = df.nearest_park.unique(),
    y = df.nearest_park.value_counts(),
    title= "Number of Listing Near the Parks",
    labels={
        'y': 'Number of Listing',
        'x': 'Park'
    },
    template='ggplot2'
)
fig.show()

In [ ]:
#df_near = df.sort_values(by = 'distance_to_nearest_park', ascending=True)
#df_near.info()
df_neig_dis = df.groupby(['neighbourhood']).agg({'distance_to_nearest_park': ['mean','std','count']})
#df_neig_dis.sort_values(by='mean')
df_neig_dis_sorted = df_neig_dis.distance_to_nearest_park.sort_values('mean').head(6)
df_neig_dis_sorted = pd.DataFrame(df_neig_dis_sorted)
df_neig_dis_sorted.index

In [ ]:


fig = px.bar(
    df_neig_dis_sorted,
    x = df_neig_dis_sorted.index,
    y = 'mean',
    color = 'count',
    title= "Mean distance of nearest park organized by neighbourhood.",
    labels={
        'mean': 'Mean distance of nearest park (km)',
        'neighbourhood': 'Neighourhoods'
    }
)
fig.show()

# The Map Code

In [ ]:
#center the map around montreal
map_osm = folium.Map(location=[df['latitude'].mean() , df['longitude'].mean()] , zoom_start=10)
temp = df
#adding a title
title_html = '''
             <h3 align="center" style="font-size:20px"><b>Concetration of listing and park locations.</b></h3>
             '''
map_osm.get_root().html.add_child(folium.Element(title_html))
#make the listing location as list
location = list(zip(df.latitude, df.longitude))
folium.plugins.HeatMap(location, min_opacity = 0.09, max_opacity=0.5).add_to(map_osm)

In [ ]:
#make the park's location as list
obs = list(zip(df3['Latitude'], df3['Longitude']))
park_list = ['Park 1', "Park 2", "Park 3", 'Park 4']

for i , el in enumerate(obs):
    #folium.CircleMarker(el[0:2], radius=15, color='#000000', fill_color='#000000').add_to(map_osm)
    folium.Marker(el, popup=park_list[i], icon=folium.Icon(icon='tree', prefix= 'fa', color='green')).add_to(map_osm)
#FastMarkerCluster(data=obs).add_to(map_osm)



In [ ]:
map_osm